# Jane Street: Neural Network Starter

I try implementing a simple Tensorflow Keras neural network here. Train in Version 17.

**Caution:** The GroupCV method applied in this notebook may cause time leakage problem. Please use [Purged Time-Series CV][1] instead.

[1]: https://www.kaggle.com/marketneutral/purged-time-series-cv-xgboost-optuna

In [1]:
TRAINING = True

In [2]:
class LiteModel:
    
    @classmethod
    def from_file(cls, model_path):
        return LiteModel(tf.lite.Interpreter(model_path=model_path))
    
    @classmethod
    def from_keras_model(cls, kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return LiteModel(tf.lite.Interpreter(model_content=tflite_model))
    
    def __init__(self, interpreter):
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        input_det = self.interpreter.get_input_details()[0]
        output_det = self.interpreter.get_output_details()[0]
        self.input_index = input_det["index"]
        self.output_index = output_det["index"]
        self.input_shape = input_det["shape"]
        self.output_shape = output_det["shape"]
        self.input_dtype = input_det["dtype"]
        self.output_dtype = output_det["dtype"]
        
    def predict(self, inp):
        inp = inp.astype(self.input_dtype)
        count = inp.shape[0]
        out = np.zeros((count, self.output_shape[1]), dtype=self.output_dtype)
        for i in range(count):
            self.interpreter.set_tensor(self.input_index, inp[i:i+1])
            self.interpreter.invoke()
            out[i] = self.interpreter.get_tensor(self.output_index)[0]
        return out
    
    def predict_single(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        inp = np.array([inp], dtype=self.input_dtype)
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out[0]
    
    def predict_single2(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out
    
    def predict_lstm(self, inp):
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        result = self.interpreter.get_tensor(self.output_index)
        return result[0,-1]

In [3]:
import warnings
warnings.filterwarnings('ignore')

#if TRAINING:
#    import cudf
#    import cupy as cp

import os, gc
import pandas as pd
import numpy as np
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from joblib import dump, load

import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental.preprocessing import Normalization


import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args


import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
import kerastuner as kt

In [4]:
TARGET = 'action'
FEATS = ['feature_{}'.format(int(i)) for i in range(130)]
SEED = 42
FOLDS = 5

In [5]:
def create_autoencoder(input_dim,output_dim,noise=0.1):
    i = Input(130)
    mask = Input(130)
    encoded = BatchNormalization()(i)
    encoded = GaussianNoise(noise)(encoded)
    
    encoded = Dense(96, activation = 'elu')(encoded)
    encoded = Dense(64,activation='linear')(encoded)
    encoder = Model(inputs=i,outputs=encoded)
    
    return encoder

In [6]:
encoder = create_autoencoder(130, 5, noise=0.1)
encoder.load_weights('../input/js-cv-split2/encoder.hdf5')
encoder.trainable = False

In [7]:
def create_model1(input_dim,output_dim):
    inputs = Input(input_dim)
    
    #x = encoder(inputs)
    #x = Concatenate()([x,inputs]) #use both raw and encoded features
    x = BatchNormalization()(inputs)
    
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0)(x)    
    
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0)(x)
    
    x = Dense(300)(x)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0)(x)
    
    
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0)(x)
    
    x = Dense(output_dim,activation='sigmoid', name = 'label_output')(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model

In [8]:


def create_model2(input_dim,output_dim):
    inputs = Input(input_dim)
    
    #x = encoder(inputs)
    #x = Concatenate()([x,inputs]) #use both raw and encoded features
    x = BatchNormalization()(inputs)

    x = Dense(438)(x)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0)(x)

    x = Dense(420)(x)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0)(x)

    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Lambda(tf.keras.activations.swish)(x)
    x = Dropout(0)(x)
    

    x = Dense(output_dim,activation='sigmoid')(x)
    model = Model(inputs=inputs,outputs=x)
    model.compile(optimizer=Adam(0.0072342),loss=BinaryCrossentropy(label_smoothing=0.090004),metrics=[tf.keras.metrics.AUC(name = 'auc')])
    return model



In [9]:
models = []

model = create_model2(130 + 64 + 88, 5)
model.load_weights(f'../input/js-non-of-model-w-sae/model2_3.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)
    
model = create_model1(130 + 64 + 88, 5)
model.load_weights(f'../input/js-non-of-model-w-sae/model_4.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)
        
model = create_model1(130 + 64 + 88, 5)
model.load_weights(f'../input/js-non-of-model-w-sae/model_5.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)        

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Input, Multiply, Add, Concatenate
from tensorflow.keras.activations import sigmoid, relu
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import AUC
from tensorflow.keras.losses import binary_crossentropy
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.activations import softmax


def build_tabnet_model(input_dim, transform_dim, N_a, N_d, num_decision_step, gamma, output_dim, B_v, m_d, Lambda, multiplier):
    # Initialization
    inputs = Input(input_dim)
    #x2 = encoder(x1)
    #x = Concatenate()([x1, x2])
    #P = tf.ones([tf.shape(x)[0], tf.shape(x)[1]]) #
    #d_out = tf.zeros([tf.shape(x)[0], N_d])
    #entropy = 0
    # pre-encoding
    
    x = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(inputs)
    feat_trans1 = Dense(transform_dim * 2, use_bias= False)(x)
    feat_trans1 = Dropout(0)(feat_trans1)
    feat_trans1 = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(feat_trans1)
    feat_trans1 = Multiply()([feat_trans1[:,:transform_dim], sigmoid(feat_trans1[:,transform_dim:])])
    
    feat_trans2 = Dense(transform_dim * 2, use_bias = False)(feat_trans1)
    feat_trans2 = Dropout(0)(feat_trans2)
    feat_trans2 = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(feat_trans2)
    feat_trans2 = Multiply()([feat_trans2[:,:transform_dim], sigmoid(feat_trans2[:,transform_dim:])])
    feat_trans2 = Add()([feat_trans2, feat_trans1])
    feat_trans2 *= tf.math.sqrt(0.5)
    
    feat_trans3 = Dense(transform_dim * 2, use_bias = False)(feat_trans2)
    feat_trans3 = Dropout(0)(feat_trans3)
    feat_trans3 = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(feat_trans3)
    feat_trans3 = Multiply()([feat_trans3[:,:transform_dim], sigmoid(feat_trans3[:,transform_dim:])])
    feat_trans3 = Add()([feat_trans3, feat_trans2])
    feat_trans3 *= tf.math.sqrt(0.5)
    
    feat_trans4 = Dense(transform_dim * 2, use_bias = False)(feat_trans3)
    feat_trans4 = Dropout(0)(feat_trans4)
    feat_trans4 = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(feat_trans4)
    feat_trans4 = Multiply()([feat_trans4[:,:transform_dim], sigmoid(feat_trans4[:,transform_dim:])])
    feat_trans4 = Add()([feat_trans4, feat_trans3])
    feat_trans4 *= tf.math.sqrt(0.5)
    
    for step in range(num_decision_step):
        #Attentive transform
        
        mask_value = Dense(input_dim, use_bias= False)(feat_trans4[:,-N_a:])
        mask_value = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(mask_value)
        if step == 0:
            mask_value = softmax(multiplier * mask_value)
            P = (gamma - mask_value)
        else:
            mask_value *= P
            mask_value = softmax(multiplier * mask_value)
            P *= (gamma - mask_value)

        masked_feature = Multiply()([mask_value, inputs])
        
        
        # Entropy is used to penalize the amount of sparsity in feature
        # selection.
        #entropy += tf.reduce_mean(tf.reduce_sum(-mask_value * tf.math.log(mask_value + 0.01), axis=1)) / (num_decision_step)
        
        
        # feature_transform
        feat_trans1 = Dense(transform_dim * 2, use_bias= False)(masked_feature)
        feat_trans1 = Dropout(0)(feat_trans1)
        feat_trans1 = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(feat_trans1)
        feat_trans1 = Multiply()([feat_trans1[:,:transform_dim], sigmoid(feat_trans1[:,transform_dim:])])
    
        feat_trans2 = Dense(transform_dim * 2, use_bias = False)(feat_trans1)
        feat_trans2 = Dropout(0)(feat_trans2)
        feat_trans2 = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(feat_trans2)
        feat_trans2 = Multiply()([feat_trans2[:,:transform_dim], sigmoid(feat_trans2[:,transform_dim:])])
        feat_trans2 = Add()([feat_trans2, feat_trans1])
        feat_trans2 *= tf.math.sqrt(0.5)
    
        feat_trans3 = Dense(transform_dim * 2, use_bias = False)(feat_trans2)
        feat_trans3 = Dropout(0)(feat_trans3)
        feat_trans3 = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(feat_trans3)
        feat_trans3 = Multiply()([feat_trans3[:,:transform_dim], sigmoid(feat_trans3[:,transform_dim:])])
        feat_trans3 = Add()([feat_trans3, feat_trans2])
        feat_trans3 *= tf.math.sqrt(0.5)
        
        feat_trans4 = Dense(transform_dim * 2, use_bias = False)(feat_trans3)
        feat_trans4 = Dropout(0)(feat_trans4)
        feat_trans4 = BatchNormalization(virtual_batch_size=B_v, momentum = m_d)(feat_trans4)
        feat_trans4 = Multiply()([feat_trans4[:,:transform_dim], sigmoid(feat_trans4[:,transform_dim:])])
        feat_trans4 = Add()([feat_trans4, feat_trans3])
        feat_trans4 *= tf.math.sqrt(0.5)
        
        # ouput
        if step == 0:
            d_out = relu(feat_trans4[:,:N_d])
        else:
            d_out += relu(feat_trans4[:,:N_d])
        
    y1 = Dense(output_dim, activation = 'sigmoid', name = 'label_out')(d_out)
    
    model = Model(inputs = inputs, outputs = y1)
    return model

In [11]:
model = build_tabnet_model(130 + 64 + 88, 32, 16, 16, 3, 1.2, 5, None, 0.8, 0, 6)
model.load_weights(f'../input/js-non-of-model-w-sae/tabnet_model_42_3_multiplier_6_version2.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)

model = build_tabnet_model(130 + 64 + 88, 32, 16, 16, 3, 1.2, 5, None, 0.8, 0, 4)
model.load_weights(f'../input/js-non-of-model-w-sae/tabnet_model_42_4_multiplier_4_version4.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)

model = build_tabnet_model(130 + 64 + 88, 32, 16, 16, 3, 1.2, 5, None, 0.8, 0, 4)
model.load_weights(f'../input/js-non-of-model-w-sae/tabnet_model_42_5_multiplier_4_version8.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)

In [12]:
from tensorflow.keras.layers import Conv1D, AveragePooling1D, MaxPooling1D, Flatten, Multiply
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Input, Multiply, Add, Concatenate, Softmax
from tensorflow.keras.activations import sigmoid, relu
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import AUC
from tensorflow.keras.losses import binary_crossentropy
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.activations import softmax

In [13]:


def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
      q: query shape == (..., seq_len_q, depth)
      k: key shape == (..., seq_len_k, depth)
      v: value shape == (..., seq_len_v, depth_v)
      mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
      output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = Softmax()(scaled_attention_logits)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights



In [14]:
def transformer_model(input_dim1, window_size, d_model, rate, num_layers, dff, num_heads, output_dim):
    inputs = Input(input_dim1)
    batch_size = tf.shape(inputs)[0]
    #mask = create_look_ahead_mask(seq)
    x = BatchNormalization()(inputs)
    x = Dense(d_model * window_size, use_bias = False)(x)
    x = tf.reshape(x, (batch_size, window_size, d_model))
    
    #x *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    #x += positional_encoding(seq, d_model)
    #x = Dropout(rate)(x)
    
    for i in range(num_layers):
        
        #multihead_attention
        q = Dense(d_model)(x)
        k = Dense(d_model)(x)
        v = Dense(d_model)(x)
        
        q = tf.reshape(q, (batch_size, -1, num_heads, d_model//num_heads))
        q = tf.transpose(q, perm=[0, 2, 1, 3])
        
        k = tf.reshape(k, (batch_size, -1, num_heads, d_model//num_heads))
        k = tf.transpose(k, perm=[0, 2, 1, 3])
        
        v = tf.reshape(v, (batch_size, -1, num_heads, d_model//num_heads))
        v = tf.transpose(v, perm=[0, 2, 1, 3])
        
        attn_out, _ = scaled_dot_product_attention(q, k, v, None)
        attn_out = tf.transpose(attn_out, perm=[0, 2, 1, 3])  
        attn_out = tf.reshape(x, (batch_size, -1, d_model))  
        attn_out = Dense(d_model)(attn_out)  
        attn_out = Dropout(rate)(attn_out)
        attn_out = BatchNormalization()(attn_out + x)
        
        fnn_out = Dense(dff, activation= 'relu')(attn_out)  # (batch_size, seq_len, dff)
        fnn_out = Dense(d_model)(fnn_out)
        fnn_out = Dropout(rate)(fnn_out)
        x = BatchNormalization()(fnn_out + attn_out)
        if i <= 1:
            x = AveragePooling1D(2)(x)
        else:
            x = MaxPooling1D(2)(x)
    
    x = Dense(64)(x)
    x = Flatten()(x)
    #x = BatchNormalization()(x)
    x = Dropout(rate)(x)
    x = Dense(output_dim, activation = 'sigmoid', name = 'label_out')(x)
    model = Model(inputs = inputs, outputs = x)    
    return model

In [15]:
def transformer_model2(input_dim1, window_size, d_model, rate, num_layers, dff, num_heads, output_dim):
    inputs = Input(input_dim1)
    batch_size = tf.shape(inputs)[0]
    #mask = create_look_ahead_mask(seq)
    x = BatchNormalization()(inputs)
    x = Dense(d_model * window_size, use_bias = False)(x)
    x = tf.reshape(x, (batch_size, window_size, d_model))
    
    #x *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    #x += positional_encoding(seq, d_model)
    #x = Dropout(rate)(x)
    
    for i in range(num_layers):
        
        #multihead_attention
        q = Dense(d_model)(x)
        k = Dense(d_model)(x)
        v = Dense(d_model)(x)
        
        q = tf.reshape(q, (batch_size, -1, num_heads, d_model//num_heads))
        q = tf.transpose(q, perm=[0, 2, 1, 3])
        
        k = tf.reshape(k, (batch_size, -1, num_heads, d_model//num_heads))
        k = tf.transpose(k, perm=[0, 2, 1, 3])
        
        v = tf.reshape(v, (batch_size, -1, num_heads, d_model//num_heads))
        v = tf.transpose(v, perm=[0, 2, 1, 3])
        
        attn_out, _ = scaled_dot_product_attention(q, k, v, None)
        attn_out = tf.transpose(attn_out, perm=[0, 2, 1, 3])  
        attn_out = tf.reshape(x, (batch_size, -1, d_model))  
        attn_out = Dense(d_model)(attn_out)  
        attn_out = Dropout(rate)(attn_out)
        attn_out = BatchNormalization()(attn_out + x)
        
        fnn_out = Dense(dff, activation= 'relu')(attn_out)  # (batch_size, seq_len, dff)
        fnn_out = Dense(d_model)(fnn_out)
        fnn_out = Dropout(rate)(fnn_out)
        x = BatchNormalization()(fnn_out + attn_out)
        if i <= 1:
            x = MaxPooling1D(2)(x)
        else:
            x = MaxPooling1D(2)(x)
    
    x = Dense(64)(x)
    x = Flatten()(x)
    #x = BatchNormalization()(x)
    x = Dropout(rate)(x)
    x = Dense(output_dim, activation = 'sigmoid', name = 'label_out')(x)
    
    model = Model(inputs = inputs, outputs = x)    
    return model

In [16]:
model = transformer_model(130 + 64 +88, 16, 128, 0, 3, 256, 4, 5)
model.load_weights(f'../input/js-non-of-model-w-sae/transformer_no_imputer_34_version2.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)

model = transformer_model2(130 + 64 + 88, 16, 128, 0, 3, 256, 4, 5)
model.load_weights(f'../input/js-non-of-model-w-sae/transformer_no_imputer_45_version5.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)

model = transformer_model2(130 + 64 +88, 16, 128, 0, 3, 256, 4, 5)
model.load_weights(f'../input/js-non-of-model-w-sae/transformer_no_imputer_55_version4.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)

In [17]:
from tensorflow.keras.layers import Conv1D, AveragePooling1D, MaxPooling1D, Flatten, Multiply, Add

input_dim = 130 + 64 + 88
output_dim = 5
window_size = 16
cha1 = 128
cha2 = 256
rate = 0.0


def conv_model(input_dim, output_dim, window_size, cha1, cha2, rate):
    inputs = Input(input_dim)
    x = BatchNormalization()(inputs)
    x = Dense(window_size * cha1, use_bias = False)(x)
    x = tf.reshape(x, (-1, window_size, cha1))
    x = BatchNormalization()(x)
    
    for i in range(3):
        #x = BatchNormalization()(x)
        x = Dropout(rate)(x)
        x = Conv1D(cha1, 3, activation = tf.keras.activations.swish, padding = 'causal')(x)
        temp_x = x
        
        x = BatchNormalization()(x)
        x = Dropout(rate)(x)
        x = Conv1D(cha2, 3, activation = tf.keras.activations.swish, padding = 'causal')(x)

        x = BatchNormalization()(x)
        x = Dropout(rate)(x)
        x = Conv1D(cha1, 3, activation = tf.keras.activations.swish, padding = 'causal')(x)
        
        x = Add()([temp_x,x])
        x = BatchNormalization()(x)
        x = MaxPooling1D(2)(x)
        
    x = Dense(64)(x)    
    x = Flatten()(x)
    x = BatchNormalization()(x)
    x = Dropout(rate)(x)
    x = Dense(output_dim, activation = 'sigmoid', name = 'label_out')(x)

    
    model = Model(inputs =inputs, outputs = x) 
    
    return model

In [18]:
model = conv_model(input_dim, output_dim, window_size, cha1, cha2, rate)
model.load_weights(f'../input/js-non-of-model-w-sae/conv_model3_42_3.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)    

model = conv_model(input_dim, output_dim, window_size, cha1, cha2, rate)
model.load_weights(f'../input/js-non-of-model-w-sae/conv_model5_42_4.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)    

model = conv_model(input_dim, output_dim, window_size, cha1, cha2, rate)
model.load_weights(f'../input/js-non-of-model-w-sae/conv_model4_42_5.hdf5')
if not TRAINING:
    model = LiteModel.from_keras_model(model)
models.append(model)    

In [19]:
def create_model(input_dim,output_dim):
    inputs = Input(input_dim)
    
    #x = encoder(inputs)
    #x = Concatenate()([x,inputs]) #use both raw and encoded features
    x = BatchNormalization()(inputs)
    x = Dropout(0.2)(x)
    
    
    x1 = Dense(256)(x)
    x1 = BatchNormalization()(x1)
    x1 = Lambda(tf.keras.activations.swish)(x1)
    x1 = Dropout(0.2)(x1)    
    
    x2 = Concatenate()([x1, x])
    x2 = Dense(256)(x2)
    x2 = BatchNormalization()(x2)
    x2 = Lambda(tf.keras.activations.swish)(x2)
    x2 = Dropout(0.2)(x2)

    x3 = Concatenate()([x2, x1])
    x3 = Dense(256)(x3)
    x3 = BatchNormalization()(x3)
    x3 = Lambda(tf.keras.activations.swish)(x3)
    x3 = Dropout(0.2)(x3)
    
    x4 = Concatenate()([x3, x2])
    x4 = Dense(256)(x4)
    x4 = BatchNormalization()(x4)
    x4 = Lambda(tf.keras.activations.swish)(x4)
    x4 = Dropout(0.2)(x4)
    
    x = Concatenate()([x4, x3])
    
    x = Dense(output_dim,activation='sigmoid', name = 'label_output')(x)
    
    model = Model(inputs = inputs, outputs = x)
    return model

In [20]:
for f in range(3,6):
    model = create_model(130 + 64 + 88, 5)
    model.load_weights(f'../input/js-non-of-model-w-sae/resnet_model_{f}.hdf5')
    if not TRAINING:
        model = LiteModel.from_keras_model(model)
    models.append(model)    

In [21]:
if TRAINING:
    print('Loading...')
    train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
    TARGET = 'action'
    FEATS = ['feature_{}'.format(int(i)) for i in range(130)]

    print('Filling...')
    train = train.query('weight > 0').reset_index(drop = True)
    resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
    y = np.stack([(train[c] > 0.00000).astype('int') for c in resp_cols]).T
    X = train[FEATS].to_numpy()
    wr = train.weight*train['resp'].to_numpy()
    date = train['date'].values
    print('Finish.')

Loading...
Filling...
Finish.


In [22]:
nan_feat_bool = pd.read_pickle('../input/nn-model-wpt/nfb.pkl')

In [23]:
if TRAINING:
    with open('../input/js-cv-split2/f_mean.npy', 'rb') as f:
        f_mean = np.load(f)
    mask2 = np.isnan(X[:,nan_feat_bool]).astype(int)
    X = np.nan_to_num(X) + np.isnan(X).astype(int) * f_mean
    del(train)
    _= gc.collect()
    splits = pd.read_pickle('../input/js-cv-split2/cross_validation.pkl')

In [24]:
if TRAINING:
    X_test = X[splits[3][1]]
    y_test = y[splits[3][1]]
    mask_test = mask2[splits[3][1]]
    encoded_X_test = encoder(X_test).numpy()
    X_test = np.concatenate((X_test, encoded_X_test, mask_test), axis = -1)
    date_test = date[splits[3][1]]
    wr_test = wr[splits[3][1]]
    del(mask_test)
    _= gc.collect()

In [25]:
def metrics(y_true, y_pred):
    Pi = np.bincount(y_true, y_pred)
    unique_y_true = np.unique(y_true).tolist()
    Pi = Pi[unique_y_true]
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / len(Pi))
    u = min(max(t, 0), 6) * np.sum(Pi)
    return u

In [26]:
if TRAINING:
    Y_hat = [model.predict(X_test, batch_size = 10000) for model in models]

In [27]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.5,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.3079814681421412 0.3585962369442555 0.07469957955632908 0.13922506800233042 0.11949764735494386 872.933826176824
0.11095917983917361 0.25053404490472136 0.025397859540341558 0.1878729886157286 0.42523592710003477 873.508226402523
0.2341097468001378 0.46163162134660723 0.04730360870332698 0.20536705740438527 0.0515879657455428 882.6217967072467
0.19154661814435237 0.4061878016074461 0.16681927165153795 0.10877895922281271 0.12666734937385077 892.0738948168512
0.22899426391519362 0.3242816239640605 0.1816588580070251 0.20607451428609763 0.05899073982762321 928.5529421265123
0.22132803641323712 0.3148501350431466 0.12792162234811133 0.2367310836790099 0.09916912251649494 929.0895322310607
0.11924184884133565 0.24785335756000873 0.023566872126319038 0.4540407864746376 0.15529713499769895 939.7511895590383
0.024129651897800593 0.26607770192368047 0.01286127537605737 0.4370120245115372 0.25991934629092434 953.784386625709
0.006605301468582483 0.2601182646840167 0.016696921315055974 0.47617

In [28]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.501,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.14917054029004165 0.1309139210168495 0.26628344660581504 0.16289031598868817 0.2907417760986057 724.4392203105293
0.20729580806632375 0.19475216226688732 0.07959328942653725 0.23195615007291281 0.2864025901673388 766.5046258135458
0.1444725095049283 0.3240085723896081 0.14862329330227955 0.128506789171127 0.25438883563205694 833.1573810114927
0.4671191751159647 0.3569821659596866 0.06114069999907623 0.06625058940553484 0.048507369519737636 885.9291366870827
0.07911064729172107 0.2519894806023556 0.042381408162448383 0.40134681384767185 0.22517165009580323 887.0214123069184
0.09069308286797786 0.12488337278706263 0.007590266651639344 0.5327916551758254 0.24404162251749478 905.8811762024894
0.3881064707617597 0.33914328105864994 0.03770624180826811 0.11963212591786675 0.1154118804534554 943.0032915672796
0.3431632697825074 0.3658442393760596 0.030233179476729877 0.09964334573450834 0.16111596563019484 969.7709045957046
0.06353153431191438 0.2436057317320512 0.018993187070052674 0.41064

In [29]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.502,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.33826383712041636 0.16063786071708633 0.0011681163503462159 0.053085434142356176 0.4468447516697948 695.8763953974496
0.14205769470732657 0.06428340765802701 0.2766750924289496 0.27073790841757406 0.24624589678812275 723.0473556439733
0.26313832143230276 0.3772585330987841 0.2554312884982978 0.07528336758912262 0.02888848938149262 783.0780824571364
0.31190354200350945 0.1102841221676306 0.144718196844057 0.42835774098910423 0.00473639799569874 856.9116949889374
0.03722081540339427 0.25184750236606906 0.3050925576958081 0.28441111393612684 0.12142801059860182 906.0698515637686
0.25663921986079263 0.4359001072089238 0.0823932973807857 0.16338653992594948 0.06168083562354858 988.0944509311948
0.32615953052668173 0.36747659100490093 0.01909092892488622 0.08901169042796755 0.1982612591155636 1003.4714678418829
0.2676303933638957 0.45971815223247287 0.004789356840239835 0.1401260528774748 0.1277360446859168 1006.1555502830299


In [30]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.503,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.2870864552551051 0.2730666693914633 0.15081285513573575 0.2188188377536565 0.07021518246403942 826.987006178599
0.07637364559330051 0.36103464476661257 0.11335831974595584 0.3716039426867591 0.07762944720737197 840.5451398475101
0.2240053650124868 0.5446254313888133 0.03352209962111323 0.1854069029277063 0.01244020104988031 842.9193508319186
0.2512576719599817 0.3495159003073284 0.12440852381799108 0.16804935734679058 0.1067685465679083 968.8464371598384
0.2894146935875779 0.48118356055589345 0.024240714214502062 0.07960801016409172 0.1255530214779349 970.9435328984451
0.3335354703725876 0.41777098063260115 0.013618281282694242 0.09023019828600386 0.14484506942611322 984.1099195984375
0.0048620476406226795 0.4440200954815811 0.2211152315754704 0.17179409760999015 0.15820852769233576 990.4850225058332
0.3775333975128857 0.36761926572060244 0.03541146576894341 0.18216145133511608 0.037274419662452325 995.6369758538366
0.3724003998512162 0.3562474588323985 0.004613115094585114 0.1284852

In [31]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.504,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.25466163543435627 0.05528576748543068 0.2353585535537639 0.19375545242193287 0.2609385911045163 736.2616372798985
0.24073142404969042 0.35532979968386424 0.15356056065721718 0.13957496021274063 0.11080325539648742 873.0880723113917
0.02568835281919608 0.2985791297205483 0.18952906180420992 0.21588165196649486 0.2703218036895508 874.3350759009555
0.2336835851445841 0.20335367264393403 0.2929539562566683 0.23422180109213855 0.03578698486267498 917.4641544511502
0.4525647159012136 0.3486671180077987 0.032353829373500065 0.12345450523261219 0.042959831484875435 981.5560731771416
0.32369511397752193 0.3971005273169659 0.09232598332959027 0.15465669333441648 0.032221682041505405 1013.1613457010159
0.01079416820576381 0.40135673668836375 0.11599044337804199 0.42618757656488093 0.045671075162949465 1038.9664786427097


In [32]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.505,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.15311843178700338 0.2936476000066934 0.18051765069440678 0.33733537095173755 0.03538094656015908 932.1392685910215
0.0031794692721610136 0.35036110856249375 0.3307484634828448 0.2621215118889625 0.05358944679353789 949.5107363515581
0.003546243053938017 0.4043886805864571 0.06442343912165721 0.31609557572388347 0.2115460615140642 1011.084047973103
0.06103314919951458 0.3991548602643519 0.20976122963818025 0.2948109162616725 0.03523984463628075 1024.319575670113
0.05155880996882068 0.3461826231608078 0.124673800944752 0.387105185848972 0.09047958007664733 1041.6531438752754
0.00985664493011903 0.25561527590278654 0.2185629719361171 0.47615507057652023 0.039810036654457044 1049.9019349953269
0.09885080386179931 0.43468437704605567 0.16473667803926217 0.2841984007057407 0.017529740347142126 1078.5490184716232


In [33]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.506,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.13098929618616464 0.13548585225839024 0.23643257226207487 0.20279759150685542 0.2942946877865149 841.8325984209658
0.20532079077841073 0.19201485722040582 0.08801752419909248 0.2848348211501303 0.22981200665196072 867.2580815017523
0.17101337357622626 0.21812675180599167 0.1948423448909018 0.26535024627058534 0.15066728345629496 935.2036201192458
0.17049786647896084 0.19952095508201312 0.20136024054403406 0.20635705950439506 0.22226387839059697 938.8313292995929
0.08944704770662282 0.2075414835765661 0.18211787920223105 0.26728613152888836 0.25360745798569156 946.1920199073721
0.010254867624692474 0.2578709565338559 0.27213315906179225 0.27380576607179274 0.18593525070786662 948.1585528734968
0.012347109167177898 0.31500220702262194 0.3080704546150834 0.29620560008118857 0.06837462911392816 968.6360217114715
0.012525750347663058 0.43097436686640417 0.2325368520122191 0.1998181265999881 0.12414490417372567 1041.4452817014749
0.06449291778529669 0.3786589404587806 0.016347353810018252 

In [34]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.507,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.19556356094480937 0.1844217967708268 0.17197122135442794 0.17942666816206543 0.26861675276787034 905.7669536926393
0.20545061344595072 0.18025120829390828 0.16547289244236957 0.189225930935919 0.2595993548818524 934.8886705158877
0.2216578882551564 0.09646445879343445 0.3050646190278421 0.2855913953344079 0.09122163858915916 947.1473683874789
0.06428850972056435 0.1593221552705153 0.06956604212994295 0.3821521721448483 0.32467112073412907 979.0731947060775
0.06648914160262562 0.25879680916048997 0.25654922896378957 0.27376385975243006 0.14440096052066478 992.3286997320496
0.050368418012383745 0.3581395953371871 0.18849990052679763 0.32363173936962486 0.07936034675400659 1051.148221095833
0.03699048053287422 0.42986004056950566 0.022594429700635035 0.33540177490441647 0.1751532742925685 1115.8687361989496
0.0998567819881376 0.3907571030857901 0.03142220034540433 0.3522633652953465 0.12570054928532134 1123.4719679034488
0.013737419264201585 0.37750438133448844 0.17105036607159566 0.364

In [35]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat[0] + x2 * Y_hat[3] + x3*Y_hat[6] + x4 * Y_hat[9] + x5 * Y_hat[12], axis = 1) > 0.508,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.08207532029060455 0.40096730584640017 0.2913587312525899 0.2034351417235334 0.022163500886871954 936.2983715246594
0.15528565345725925 0.22477740672681593 0.11704407763956914 0.32014069734266376 0.1827521648336919 966.9090001806012
0.022793048982704452 0.3392772574759318 0.23325079088251346 0.3535170673922582 0.05116183526659199 993.0196015516572
0.12392127964390116 0.23978892195977275 0.2064035098575557 0.22576612940071528 0.2041201591380551 1053.1508091570336
0.09759653409938171 0.37444943373928735 0.1843319108937221 0.3150514368523234 0.028570684415285326 1067.8988230318164
0.08796270527935027 0.3518220719855786 0.0650646233580812 0.3874215930595991 0.10772900631739091 1159.6497276200662
0.05358130974350695 0.31044236566404676 0.08635804236310539 0.37720073100161966 0.17241755122772115 1180.5360890948848
0.006835959340400252 0.3741600596815421 0.04456681554693292 0.43450724567900684 0.139929919752118 1231.7174379811972
0.03276884157912981 0.36037269935827626 0.05838331774996787 0.

In [36]:
if TRAINING:
    X_test = X[splits[4][1]]
    y_test = y[splits[4][1]]
    mask_test = mask2[splits[4][1]]
    encoded_X_test = encoder(X_test).numpy()
    X_test = np.concatenate((X_test, encoded_X_test, mask_test), axis = -1)
    date_test = date[splits[4][1]]
    wr_test = wr[splits[4][1]]
    del(mask_test)
    _= gc.collect()

In [37]:
if TRAINING:
    Y_hat1 = [model.predict(X_test, batch_size = 10000) for model in models]

In [38]:
metrics(date_test,wr_test*np.where(np.median(Y_hat1[1], axis = 1) > 0.5,1,0) )

2321.3879695751125

In [39]:
metrics(date_test,wr_test*np.where(np.median(Y_hat1[4], axis = 1) > 0.5,1,0) )

1846.9583449904496

In [40]:
metrics(date_test,wr_test*np.where(np.median(Y_hat1[7], axis = 1) > 0.5,1,0) )

1902.458712820669

In [41]:
metrics(date_test,wr_test*np.where(np.median(Y_hat1[10], axis = 1) > 0.5,1,0) )

2097.7398435383675

In [42]:
metrics(date_test,wr_test*np.where(np.median(Y_hat1[13], axis = 1) > 0.5,1,0) )

2405.2072661476536

In [43]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.5,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.08568570496878967 0.22159493732295524 0.3443871336881984 0.02254184277617464 0.32579038124388204 2537.549049734963
0.049030094215569454 0.2555344451384079 0.2635954756983567 0.1608354307653561 0.2710045541823099 2543.300375243085
0.10713594630676368 0.18520183740934587 0.06397514878417979 0.2857234287602672 0.35796363873944337 2549.242932028821
0.04477805071749724 0.344106624723414 0.07834708649148933 0.06773903186206348 0.46502920620553595 2561.500157275914
0.15298067499891252 0.4178185562047195 0.11871374601549707 0.07412554173720985 0.23636148104366106 2597.2446825448233
0.04523437141342972 0.2218309309002266 0.1930094231265177 0.4355420986249902 0.10438317593483583 2616.4320846845326
0.08449553987162234 0.23259402899641748 0.005004091430549112 0.35237663642365896 0.3255297032777521 2622.304635766675
0.05396558854251107 0.2709387912511713 0.04671478646710351 0.22209067808532043 0.40629015565389365 2650.25358398811
0.03017071696054113 0.29610736643028196 0.0865160147192249 0.233868

In [44]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.501,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.10455343308353689 0.04580130748271621 0.27777825983969956 0.34509053064908213 0.22677646894496525 2388.042683908332
0.1818995275776406 0.37016190452726433 0.13511516873190405 0.13807653129560354 0.17474686786758745 2593.606150198506
0.2576082040121386 0.3567663550210745 0.10128041164495344 0.022840650893480422 0.26150437842835295 2662.209174813335
0.04614234110907472 0.31103416802721734 0.10284994211891167 0.061407699509647765 0.4785658492351485 2669.977218481924
0.21208146599376546 0.3475103543933038 0.04206309167856903 0.026655339931599385 0.37168974800276217 2688.789951457881
0.05739893683307254 0.2840789582408743 0.027867982852911463 0.1866803711483003 0.44397375092484137 2705.6231507850325
0.09687047876227639 0.22446418947254634 0.061058716182328075 0.376955321140661 0.24065129444218822 2712.1172320660307
0.15121992875086232 0.18553512508020267 0.0035506000106211395 0.25271502855531225 0.4069793176030016 2744.9514488554273
0.013533454445791303 0.24389604588868305 0.0014752446710

In [45]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.502,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.2804259822235328 0.2778015319094446 0.10008467026958583 0.27298671891663934 0.0687010966807974 2628.2035381170317
0.13768100964479996 0.279199551247128 0.11508484604453084 0.23946800945587324 0.22856658360766804 2668.2243728082344
0.026166239287986344 0.3212472920274113 0.13948851428899353 0.1899999553224785 0.32309799907313036 2673.1509639558635
0.04951320538977274 0.2979197743819351 0.12913512295733837 0.3717738602198925 0.15165803705106126 2680.1463267647905
0.033066848209893175 0.26219478099653837 0.05014390194747806 0.3329900230182685 0.3216044458278219 2698.1305565909943
0.05354990171689396 0.28716382976456784 0.04795753476286782 0.31080858390693805 0.3005201498487324 2719.0139089352224
0.03588001372367669 0.0637332538304938 0.009257561696021419 0.748738217437927 0.14239095331188095 2725.0489157351954
0.16138158680714004 0.18861789407043686 0.020901934563423482 0.2746947632310963 0.3544038213279033 2729.5607390688792
0.02697278721860478 0.2400014960468074 0.1838457346699918 0.5

In [46]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.503,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.08571439829140874 0.20665445628401566 0.29972215671164293 0.2606591512811964 0.14724983743173622 2554.656362698051
0.4520022883144757 0.10060774726109707 0.03059515776494642 0.24185822774664026 0.17493657891284062 2595.306317798908
0.196666043143126 0.3299653122364915 0.08944076491385317 0.1583745557424978 0.22555332396403155 2609.547549727412
0.12222748606041066 0.18095664051992494 0.23288931678285776 0.2854254746019214 0.17850108203488524 2619.070408634136
0.22954843099983258 0.2997161172455836 0.1924129070424231 0.03495974606058636 0.24336279865157434 2646.7702981931416
0.08572813731702865 0.24409754825091454 0.077434697584423 0.30621358128516374 0.28652603556247014 2662.663985547515
0.20533366700829062 0.24466799274685336 0.18656504382270672 0.2006288864448072 0.1628044099773421 2671.1750771479064
0.08249262223434403 0.3095414687496054 0.17336836818592308 0.16993345501084728 0.2646640858192802 2672.178361164505
0.12119399144346504 0.26386847187372736 0.16177339501322235 0.2524240

In [47]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.504,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.291314465543011 0.2179320943036982 0.16788296225255775 0.10942221538675383 0.21344826251397922 2658.716445129439
0.14531687375557253 0.30934682762376864 0.28200977899739016 0.21910241914047993 0.044224100482788554 2691.5090708984335
0.06768525078873892 0.2686138005719646 0.25916690372322904 0.32728370681374686 0.07725033810232054 2699.6891627738346
0.18757624501427705 0.2943640840830242 0.17966124906047728 0.2152735841578134 0.12312483768440804 2709.2597424723085
0.24213989907938205 0.2773380327893664 0.21946402020351777 0.1621602495323349 0.09889779839539894 2724.326144042841
0.16956317623130743 0.2536383951272364 0.10157429375050145 0.350041948032364 0.1251821868585907 2727.80087980317
0.02268858220361428 0.2993635834268461 0.17486586168546076 0.31733825883019484 0.18574371385388405 2729.804435469823
0.10818873562767169 0.3406683163675738 0.2658396258681098 0.24774920686859592 0.03755411526804881 2733.248424245415
0.18364460943043365 0.28906609797149685 0.15673036552328037 0.235554

In [48]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.505,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.2192538168822125 0.2158253645925626 0.07732712630495696 0.30348497467908114 0.18410871754118682 2565.010196648147
0.3441752539233702 0.2042523863587745 0.09354888213973678 0.17591729269703898 0.18210618488107966 2593.445600715115
0.2689741388850393 0.18903735209066783 0.2535362628846275 0.25979440428952005 0.028657841850145353 2640.4871367442556
0.14324083118765163 0.025344434570528238 0.1719363173600825 0.49409990355871636 0.16537851332302123 2661.0803502075573
0.06976115675624771 0.3083452184318285 0.24593394995300663 0.2423401864346448 0.13361948842427238 2705.2909066743277
0.25091111673650945 0.18449749013723998 0.18314162929364536 0.23543651238772348 0.1460132514448819 2705.4001640962583
0.07956513628804879 0.30602107297666914 0.19228933148430366 0.3234283805139474 0.09869607873703105 2784.307351613931


In [49]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.506,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.2761120519129535 0.15307289511571043 0.1953707681436863 0.29875370736795936 0.07669057745969028 2714.2277905731753
0.172915962467998 0.27747422020765 0.21284387703545665 0.2993031409373591 0.037462799351536215 2728.556978743539
0.2739699308122337 0.15477209260943636 0.1872111928030275 0.30505617466913276 0.07899060910616981 2734.2876150435245
0.008886392306777438 0.22280964561710392 0.30181844030533933 0.38154374812386116 0.0849417736469181 2762.9883405687488
0.08065502963845621 0.3491133534518985 0.42901841021077614 0.009737879078925335 0.1314753276199438 2769.461668498222
0.035520859337521776 0.19736966343118623 0.3215215982822599 0.4302988648056214 0.015289014143410686 2780.337198927751
0.214030786304472 0.022094579740217297 0.10707772520618768 0.531857529052126 0.12493937969699703 2793.7308946190365
0.0313966987398687 0.22848079110607275 0.2981212528319303 0.4371892107289819 0.004812046593146281 2807.7182339829087


In [50]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.507,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.10824889849302285 0.12304701057944761 0.09110370355164943 0.32712974072016404 0.35047064665571603 2614.0294494810128
0.14854430884892297 0.27147077775180894 0.30566022533521714 0.10078033870368024 0.1735443493603707 2667.681389351309
0.0519357241010344 0.1991023315398782 0.20683501397917195 0.3973565413420273 0.1447703890378881 2714.238790270394
0.0848298915450175 0.2854834692324737 0.222210794762085 0.32646740620063003 0.08100843825979383 2750.3821401088353
0.021347920221826237 0.36925382640279897 0.386919924919902 0.15795908087123423 0.06451924758423865 2763.105399983616
0.03004440443620463 0.2914370590809452 0.253581570884835 0.33426496585697985 0.09067199974103533 2766.732463396421
0.014544421493933469 0.2487119887193557 0.23336608659997604 0.45590599456786085 0.047471508618873955 2774.120875172632
0.029560044776829348 0.30290888113043424 0.45891826106920797 0.05689633753967535 0.15171647548385295 2780.5423232110043
0.11449038531322989 0.07598575897339331 0.03458717044947464 0.62

In [51]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat1[1] + x2 * Y_hat1[4] + x3*Y_hat1[7] + x4 * Y_hat1[10] + x5 * Y_hat1[13], axis = 1) > 0.508,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.28070870663088326 0.05228724634084075 0.18150218989367348 0.1973202396371849 0.28818161749741766 2619.9882164174865
0.06306636860469422 0.25487940553362204 0.4617527991099903 0.01141589410629564 0.20888553264539791 2665.2795373979357
0.2958652582019958 0.11474818299947372 0.026676595108543203 0.5222440158761352 0.040465947813852106 2710.8872664384726
0.06352612974150446 0.2470695063442028 0.22866370532687938 0.25999916417756047 0.20074149440985292 2712.92896788352
0.08932174326127115 0.23756664303762348 0.3458663141876729 0.30310158892963546 0.024143710583796957 2721.7571570468886
0.1787796523959105 0.1953301161556412 0.35817901503881583 0.16945184165697533 0.0982593747526572 2741.079487710596
0.5276292653089517 0.138888320864704 0.04190319694699617 0.1097983066539432 0.18178091022540485 2746.257087722827
0.06109058063437465 0.18517729246687906 0.3416423501416211 0.4065901026115831 0.005499674145542164 2767.4699458270807
0.028629470128084346 0.21529822910665133 0.3921691246365185 0.3

In [52]:
if TRAINING:
    X_test = X[splits[5][1]]
    y_test = y[splits[5][1]]
    mask_test = mask2[splits[5][1]]
    encoded_X_test = encoder(X_test).numpy()
    X_test = np.concatenate((X_test, encoded_X_test, mask_test), axis = -1)
    date_test = date[splits[5][1]]
    wr_test = wr[splits[5][1]]
    del(mask_test)
    _= gc.collect()

In [53]:
if TRAINING:
    Y_hat2 = [model.predict(X_test, batch_size = 10000) for model in models]

In [54]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.5,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.03785834332136613 0.1761775826833413 0.0011825919876961447 0.7087094021883488 0.07607207981924767 2672.4242990834955
0.024733674041616062 0.3072700665175018 0.21525025023452618 0.3338238181034694 0.11892219110288645 2690.9395270395617
0.21948713034203687 0.2758469005472382 0.2958910244329384 0.01729494082359531 0.19148000385419123 2792.1990198391054
0.3393488619037036 0.39440643206072395 0.1110542005555626 0.030230719152800067 0.12495978632720978 2915.022415477369
0.2611291516510158 0.18134789508035554 0.24621902176229465 0.030082758674144193 0.2812211728321899 2919.9759616282104
0.11789846124978075 0.2176670638899718 0.28246069387471406 0.011796780313462438 0.370177000672071 2949.8119040556826
0.0302829981878977 0.5846733537087901 0.3674415245594556 0.011777762150239067 0.0058243613936174695 2960.4469852707216
0.17272352127028048 0.24987977952677692 0.23071434983789788 0.0168527390613602 0.32982961030368463 2969.3021891352105
0.021887358294148694 0.6282214172091465 0.071145602116646

In [55]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.501,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.14986398124756 0.24781344700808386 0.309372656808532 0.0863954836025323 0.20655443133329177 2766.5373419014727
0.15686679368875214 0.3008791818756548 0.24893428482357302 0.039596036443112756 0.25372370316890724 2789.6989989495714
0.21257181524003477 0.17565338134316166 0.02983246074646476 0.2896993689573741 0.29224297371296465 2797.144826280186
0.2438439066512992 0.18883653757882624 0.2393011109082113 0.13485847176623622 0.19315997309542698 2894.9976502575596
0.08062709467275953 0.3856061670433609 0.03092119380856217 0.10103381097604211 0.40181173349927535 2908.9004564616757
0.2538852883030878 0.5707127443800082 0.01753822016073478 0.03337344223045992 0.12449030492570948 3081.318917505326
0.3289768360520656 0.5262581261611874 0.0746972092410145 0.006819396151533591 0.06324843239419883 3095.847326602637
0.002180593617836032 0.5259772505505153 0.000476500288030846 0.02933461485336254 0.4420310406902554 3232.0073132879834


In [56]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.502,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.22295647057193807 0.24317848875976084 0.23503270548957303 0.12511160328332468 0.17372073189540344 3042.4185429847503
0.022227905024136466 0.47953513742503007 0.10023469237888 0.023108431995932568 0.37489383317602093 3206.409963605428
0.082015034222995 0.5196898965273211 0.04189694175889528 0.023903102875812415 0.33249502461497604 3240.2543833751497


In [57]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.503,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.22620441522378926 0.15633973451282118 0.4074470702776305 0.1553066415384529 0.05470213844730614 2595.108625389187
0.029355544354748152 0.19727250467414814 0.42309414391868627 0.10648879269900523 0.2437890143534123 2796.9527329291836
0.24332374486508399 0.30229801037201787 0.18932910894158894 0.03863867718419095 0.2264104586371181 3026.6675412783075
0.15775594860874495 0.37553577087924433 0.16737729567876736 0.07144840621463079 0.2278825786186126 3064.0321641410073
0.12297456155009531 0.3069458476884532 0.08961925497418771 0.18435912376411698 0.29610121202314676 3075.7176500508785
0.044917763971694676 0.874811962865876 0.0131470334962122 0.0376479505439878 0.029475289122229197 3199.366491904247
0.0560667351069106 0.37270841465047544 0.06373569375851135 0.2072462993613014 0.3002428571228014 3202.535998803143
0.09491459509865853 0.40402657294973454 0.1427036138178248 0.017871209613259442 0.3404840085205226 3242.2539366476835
0.18201822487162958 0.546047889439398 0.03947134646754562 0.06

In [58]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.504,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.22969118566187519 0.34502788011044383 0.06652403751504145 0.3014355702829745 0.05732132642966492 3033.72868577404
0.20868522767569295 0.3243335456975266 0.08012330493268797 0.1272361975982229 0.25962172409586964 3108.324828605767
0.041066586735427305 0.5045629959390107 0.06908671702750198 0.2515058647864236 0.13377783551163658 3110.0256721002115
0.009456823910394853 0.4370292823512266 0.2818689557401931 0.04681644783809356 0.22482849016009185 3135.6513136055646
0.14774837615691874 0.2839016319903665 0.08530201666599951 0.2797586982452704 0.203289276941445 3184.4164992297938
0.1917859724325918 0.2592388406896774 0.08008676378008749 0.3088253750898702 0.1600630480077732 3198.368245347271
0.09231205584528976 0.5893646218416257 0.032096871995001906 0.11984953775005089 0.16637691256803175 3208.677621346112
0.01451735320497583 0.7243627166954343 0.13473518456575057 0.02459178892056645 0.10179295661327284 3244.401736222851
0.05264071112329308 0.3648227727285604 0.10612220863987616 0.2187985

In [59]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.505,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.07613160947765248 0.24967428279238216 0.2923478101181043 0.0854809156819609 0.2963653819299002 2993.830632341717
0.15949146496200137 0.39681260518832867 0.05150405640179265 0.30638365377911897 0.08580821966875835 3125.1535273216855
0.039727991860837836 0.013687663946080628 0.13420650861443734 0.16033377734597942 0.6520440582326649 3146.3504702425
0.20770062970770206 0.2534086028074418 0.17023449719909994 0.03752462137394075 0.3311316489118154 3151.8452789611015
0.13084027022775171 0.33166171156151836 0.024205037788842476 0.16336484727656142 0.34992813314532617 3217.6278885176985
0.1634228801949828 0.3534482918287877 0.022155435039906142 0.10312127877173521 0.3578521141645883 3315.166945268769
0.19574002030318435 0.43892511803456824 0.1394664083966662 0.19034024217090817 0.035528211094672975 3346.2743350640026
0.1690617329553966 0.5054894034098594 0.21608571274730162 0.0916198938946299 0.01774325699281248 3346.3759167062008
0.09346524934050474 0.2512504168543534 0.10876093198208181 0.

In [60]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.506,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.3946513121226355 0.001007805600766324 0.1702697141819667 0.3392285151723441 0.09484265292228741 2849.904483385868
0.2515430699385144 0.25079150496892244 0.1395513385906448 0.32478881301742557 0.03332527348449278 3167.5866285477414
0.1574096912512314 0.02865586828547698 0.0932175011148687 0.2635892223969848 0.4571277169514381 3217.670441327389
0.27624075574060014 0.31202534509157415 0.08219268039350157 0.09205336444015333 0.23748785433417072 3269.29693420388
0.1688739727689273 0.5970208478658188 0.037162154874252507 0.1608483567306689 0.0360946677603325 3319.7818078446276
0.14278826525856161 0.26038651419290515 0.11010937406546359 0.42792428140354327 0.058791565079526396 3324.557618506053
0.037611644634684464 0.4158129521455701 0.09370549691259054 0.1025296549797603 0.3503402513273945 3333.453795119127
0.018250551478219537 0.34998223270921036 0.023957271919588463 0.18570194183825373 0.42210800205472787 3363.741079723911
0.21732783540134903 0.45642324531127487 0.12441913552731271 0.186

In [61]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.507,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.05088284226868987 0.3382229415014301 0.32133895727855205 0.2824734797338145 0.007081779217513563 3155.239219949634
0.20870726346108226 0.17981557887744723 0.14071504228943116 0.22673866958132988 0.24402344579070967 3170.529058258423
0.0817357740740043 0.24369197868672776 0.3177686337107011 0.13583317233571954 0.22097044119284734 3217.8013909681777
0.17959560347350684 0.24313457278768846 0.12957058558820725 0.11137985191165213 0.33631938623894536 3308.5168571479285
0.19357817595311783 0.2649977289306031 0.13973941746718566 0.14335654690219365 0.25832813074689975 3316.92787123882
0.21348943405253828 0.20774248564813017 0.07982447567846528 0.15737464269625628 0.34156896192461006 3318.02451803312
0.016184877294030096 0.3433871488542863 0.044301586453333014 0.25583936418807895 0.3402870232102716 3325.1362153543237
0.021815556463231703 0.3060574256623016 0.08158856135513656 0.23814857650417046 0.3523898800151597 3365.6331695864305
0.07075707016241414 0.3487052068225499 0.08531705828566029 

In [62]:
if TRAINING:
    best3 = 0
    for i in range(10000):
        x1 = np.random.uniform()
        x2 = np.random.uniform()
        x3 = np.random.uniform()
        x4 = np.random.uniform()
        x5 = np.random.uniform()
        #x3 = np.random.uniform(0.48,0.52)
        x = x1 + x2 + x3 + x4 + x5
        x1 /= x
        x2 /= x
        x3 /= x
        x4 /= x
        x5 /= x
        
        score3 = metrics(date_test,wr_test*np.where(np.median(x1 * Y_hat2[2] + x2 * Y_hat2[5] + x3*Y_hat2[8] + x4 * Y_hat2[11] + x5 * Y_hat2[14], axis = 1) > 0.508,1,0) )
        if score3 > best3:
            best3 = score3
            print(x1, x2, x3, x4, x5, best3)

0.2644614780202741 0.006451869271920441 0.29477980941853005 0.20249990011595942 0.23180694317331593 3139.2669239680595
0.1845377635466259 0.0912928634413548 0.21835116447805988 0.2853090073929383 0.22050920114102118 3229.971495480074
0.09843771561419964 0.2635709874205433 0.23252970752813568 0.21543106665737508 0.19003052277974627 3273.816983236808
0.09497643498721738 0.2610039341228348 0.14626443773704403 0.18241712374228017 0.3153380694106236 3279.201264350224
0.07174711649884048 0.24529394747024713 0.1516557427689107 0.09942593011440391 0.4318772631475977 3318.8970993034904
0.0506868921724628 0.39090722359276264 0.05693327691570214 0.1490512499348997 0.3524213573841726 3410.452451403975
0.11509237129843562 0.20176096191074197 0.04473390102989092 0.28379363017745474 0.35461913558347674 3415.072794047713
0.10703622415848738 0.1580038117565612 0.0587001036111675 0.30392203078376057 0.3723378296900234 3419.802566833585
0.16504386859271813 0.20704813186644463 0.04794904185671612 0.253232

In [63]:
with open('../input/js-cv-split2/f_mean.npy', 'rb') as f:
    f_mean = np.load(f)

In [64]:
encoder = LiteModel.from_keras_model(encoder)

In [65]:
if not TRAINING:
    import janestreet
    env = janestreet.make_env()
    th = 0.51
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            #test_df[FIX_FEAT] = pt.transform(test_df[FIX_FEAT].values)
            x_tt = test_df.loc[:, FEATS].values
            mask = np.isnan(x_tt[:,nan_feat_bool]).astype(np.int8)
            if np.isnan(x_tt.sum()):

                x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt).astype(np.int8) * f_mean
                
            x_tt = x_tt.astype(np.float32)
            encoded_x = encoder.predict_single2(x_tt)
            x_tt = tf.concat([x_tt, encoded_x, mask], axis = -1)
            #0.5
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.269289583638586,0.140273060522882,0.00431820575329026,0.410287705610988,0.248819727362426,0.930825875759512,0.0745530246350113,0.28684450294532,0.0398643375719953,0.245869686115413,0.32406270916937,0.0249915809152021])
            #0.501
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.0763251457522784,0.0788625175166904,0.189492504605759,0.341901274112498,0.247725213614468,0.655959549587888,0.0187377150341519,0.240186716116674,0.143356259279585,0.56303586510107,0.433225552752167,0.0111916865267664])
            #0.502
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.0576583473691916,0.12255652240959,0.175323289611383,0.449173119701892,0.110027096104041,0.76522980391606,0.26055588039915,0.00275766092980543,0.0589500766202904,0.232612652529765,0.764658720556562,0.000496829852265645])
            #0.503
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.00870004211860127,0.0978252692956878,0.298014718855447,0.383586287383453,0.100620814834676,0.620239671826976,0.0867980001869826,0.0179772830228647,0.071241936891379,0.520915670310963,0.783576632846771,0.0105036724261971])
            #0.504
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.0900695070920216,0.0400101879000951,0.363508444081315,0.404165368235183,0.101581732824844,0.52280927277972,0.0766822336375494,0.0223235719877681,0.0813574399335528,0.429082891035245,0.836084507287292,0.0323248432054112])
            #0.505
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.14585293948777,0.0321340599888603,0.168930356391904,0.45014880680078,0.0928682653923039,0.483209333358805,0.206996385495624,0.0014347064763478,0.195922871036136,0.197001868215824,0.873562968142487,0.151937439213153])
            
            #0.506
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.123190243224635,0.0333809356163824,0.223927230959407,0.4083891129262,0.214877194263553,0.488523934898997,0.115077478026766,0.274629118750788,0.230760689860318,0.353343165822397,0.477112751369275,0.0567881442812759])
            #0.507
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.0792840481940689,0.0486631725897758,0.088028800771282,0.413007787518532,0.180887357608131,0.59477504438469,0.143226402099234,0.307389347262816,0.315100337014842,0.364481762188163,0.463060122539277,0.00209581782918519])
            #0.508
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.104034154023355,0.00943763699276361,0.0345839554383715,0.436273484206495,0.186711622897726,0.420741558536897,0.142822556506646,0.365700918525257,0.0200891010652324,0.316869805263502,0.438149821584252,0.524585384959498])
            #0.509
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.120926552448402,0.0428807234778378,0.316305249359201,0.32172273862087,0.250142935175495,0.241626069167982,0.132838781409683,0.365697181864785,0.397642473806075,0.424511927521044,0.341279159481881,0.0444262076667411])
            #0.51
            #pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [0.26694818048737,0.00541709707387954,0.283017229351308,0.378816459431733,0.266689820694482,0.269084127637271,0.0883274830502573,0.716043337455058,0.432078909025282,0.265907877030638,0.0118497447765794,0.0158197339861377])
            #test 
            pred = np.average([model.predict_single2(x_tt) for model in models], axis = 0, weights = [1]*15)
            pred = np.median(pred)
            pred_df.action = np.where(pred >= th, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df) 